# Language Modeling with Keras

In [39]:
import itertools
import numpy as np
import string
from keras.layers import Input, Embedding, merge, Flatten, Reshape, Lambda, LSTM
import keras.backend as K
from keras.models import Model

In [33]:

#imports a massive dataset of sentences found on wikipedia
with open('training_data.txt') as f:
    content = f.readlines()

# you may also want to remove whitespace characters like `\n` at the end of each line,
# also strips the punctuation
sentences = [x.strip().translate(None, string.punctuation) for x in content] 

lemma = lambda x: x.strip().lower().split(' ')
sentences_lemmatized = [lemma(sentence) for sentence in sentences]
words = set(itertools.chain(*sentences_lemmatized))
# set(['boy', 'fed', 'ate', 'cat', 'kicked', 'hat'])

# dictionaries for converting words to integers and vice versa
word2idx = dict((v, i) for i, v in enumerate(words))
idx2word = list(words)

# convert the sentences a numpy array
to_idx = lambda x: [word2idx[word] for word in x]
sentences_idx = [to_idx(sentence) for sentence in sentences_lemmatized]
# Sets the maximum word length of each sentence
max_len = 100
# If the sentence is too long, good by
for i in range(len(sentences_idx)-1, -1, -1):
    if len(sentences_idx[i]) > max_len:
        print i
        sentences_idx.pop(i)
    elif len(sentences_idx[i]) < max_len:
        sentences_idx[i] = [0] * (max_len - len(sentences_idx[i])) + sentences_idx[i]
sentences_array = np.asarray(sentences_idx, dtype='int32')
#TODO find a way to set all sentence lengths to be the same

1748


In [34]:
print len(sentences_idx)
print len(word2idx)

3148
3727


In [ ]:
n_words = len(words)
n_embed_dims = 150

# put together a model to predict
input_sentence = Input(shape=(max_len,), dtype='int32')
input_embedding = Embedding(n_words, n_embed_dims)(input_sentence)
output = LSTM(100)(input_embedding)

predict_boy = Model(input=[input_sentence], output=[output])
predict_boy.compile(optimizer='rmsprop', loss='mse')

# fit the model to predict what color each person is
predict_boy.fit([sentences_array[:len(sentences_array)-1]], [sentences_array[1:]], epochs=20, verbose=3)
embeddings = predict_boy.layers[1].get_weights()

# print out the embedding vector associated with each word
for i in range(n_words):
    print('{}: {}'.format(idx2word[i], embeddings[0][i]))

/Users/TomCat/python-neural-nets/venv/lib/python2.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=[<tf.Tenso...)`
  if __name__ == '__main__':


Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
